In [4]:
import numpy as np
import fitsio
import matplotlib.pyplot as plt
import os
import pandas as pd

In [5]:
def create_fits_TRAN(ID,files,path_file,classification):
    path = './data/'+classification+'/'
    if not os.path.isdir(path):
        os.mkdir(path)
    
    filename = './data/{}/{}.fits'.format(classification,ID)
    
    if os.path.isdir(filename):
        print("Object {} already exists!".format(ID))
        return False
    
    df = pd.read_table('../mantra_data/transient_info.txt')
    CRTS_ID = np.array(df['#CRTS ID'])
    if sum(CRTS_ID== ID) != 0:
        RA = np.array(df[CRTS_ID == ID]['RA (J2000)'])[0]
        DEC = np.array(df[CRTS_ID == ID]['Dec (J2000)'])[0]
        UT_Date = np.array(df[CRTS_ID == ID]['UT Date'])[0]
        MAG = np.array(df[CRTS_ID == ID]['Mag'])[0]
        CSS_Images = np.array(df[CRTS_ID == ID]['CSS images'])[0]
        SDSS = np.array(df[CRTS_ID == ID]['SDSS'])[0]
        OTHERS = np.array(df[CRTS_ID == ID]['Others'])[0]
        FOLLOWED = np.array(df[CRTS_ID == ID]['Followed'])[0]
        LAST = np.array(df[CRTS_ID == ID]['Last'])[0]
        LC = np.array(df[CRTS_ID == ID]['LC'])[0]
        FC = np.array(df[CRTS_ID == ID]['FC'])[0]
        CLASS = np.array(df[CRTS_ID == ID]['Classification'])[0]

        N = len(files)
        SEQUENCE = np.zeros(N).astype('S2')
        DATE = np.zeros(N, dtype='S7')
        FIELD_ID = np.zeros(N, dtype='S6')
        OBS_IN_SEQ =  np.zeros(N, dtype='S4')
        CUTOUT = np.zeros(N, dtype='S4')
        HDU_EXT = np.arange(2,N+2).astype('S4')
        for i, file in enumerate(files):
#             print(file)
            SEQUENCE[i] = file[0:2]
            DATE[i] = file[3:10]
            FIELD_ID[i] = file[11:17]
            OBS_IN_SEQ[i] = file[18:22]
            CUTOUT[i] = file[30:34]
#             print(SEQUENCE[i],DATE[i],FIELD_ID[i],OBSERVATION_IN_SEQ[i],CUTOUT[i])


        hdict = {'CRTS_ID': ID, 'RA (J2000)': RA, 'Dec (J2000)':DEC, 'N_images': N, 
                'UT Date': UT_Date, 'Mag': MAG, 'CSS images': CSS_Images,
                 'SDSS': SDSS, 'Others': OTHERS, 'Followed': FOLLOWED, 'Last': LAST,
                 'LC': LC, 'FC': FC, 'Classification': CLASS}
        data = {'HDU_Ext': HDU_EXT, 'Sequence': SEQUENCE, 'Date': DATE, 'Field_ID':FIELD_ID,
                'Obs_In_Seq': OBS_IN_SEQ, 'Cutout': CUTOUT}
        fitsio.write(filename, data, header=hdict)

        for file in files:
            img = np.load(path_file+file)
            fitsio.write(filename, img)
        return True
    else:
        print('{} Do not found in MANTRA'.format(ID))
        return False

In [6]:
classes =  ['BZ']#['AGN','BZ','CV','SN','OTHER']
for classification in classes:
    path = '../mantra_data/{}/'.format(classification)
    objects = !ls $path
    for objct in objects:
        path_file = path+objct+'/'
        files = !ls $path_file
        ID = objct[objct.find('CSS'):objct.find('CSS')+23]
        value = create_fits_TRAN(ID,files,path_file,classification)
        if value == False:
            print(path_file)   

In [ ]:
def create_fits_NON(ID,files,path_file,classification):
    path = './data/'+classification+'/'
    if not os.path.isdir(path):
        os.mkdir(path)
    
    filename = './data/{}/{}.fits'.format(classification,ID)
    
    if os.path.isdir(filename):
        print("Object {} already exists!".format(ID))
        return False
    
    df = pd.read_table('../mantra_data/transient_info.txt')
    CRTS_ID = np.array(df['#CRTS ID'])
    if sum(CRTS_ID== ID) != 0:
        RA = np.array(df[CRTS_ID == ID]['RA (J2000)'])[0]
        DEC = np.array(df[CRTS_ID == ID]['Dec (J2000)'])[0]
        UT_Date = np.array(df[CRTS_ID == ID]['UT Date'])[0]
        MAG = np.array(df[CRTS_ID == ID]['Mag'])[0]
        CSS_Images = np.array(df[CRTS_ID == ID]['CSS images'])[0]
        SDSS = np.array(df[CRTS_ID == ID]['SDSS'])[0]
        OTHERS = np.array(df[CRTS_ID == ID]['Others'])[0]
        FOLLOWED = np.array(df[CRTS_ID == ID]['Followed'])[0]
        LAST = np.array(df[CRTS_ID == ID]['Last'])[0]
        LC = np.array(df[CRTS_ID == ID]['LC'])[0]
        FC = np.array(df[CRTS_ID == ID]['FC'])[0]
        CLASS = np.array(df[CRTS_ID == ID]['Classification'])[0]

        N = len(files)
        SEQUENCE = np.zeros(N).astype('S2')
        DATE = np.zeros(N, dtype='S7')
        FIELD_ID = np.zeros(N, dtype='S6')
        OBS_IN_SEQ =  np.zeros(N, dtype='S4')
        CUTOUT = np.zeros(N, dtype='S4')
        HDU_EXT = np.arange(2,N+2).astype('S4')
        for i, file in enumerate(files):
#             print(file)
            SEQUENCE[i] = file[0:2]
            DATE[i] = file[3:10]
            FIELD_ID[i] = file[11:17]
            OBS_IN_SEQ[i] = file[18:22]
            CUTOUT[i] = file[30:34]
#             print(SEQUENCE[i],DATE[i],FIELD_ID[i],OBSERVATION_IN_SEQ[i],CUTOUT[i])


        hdict = {'CTRS_ID': ID, 'RA (J2000)': RA, 'Dec (J2000)':DEC, 'N_images': N, 
                'UT Date': UT_Date, 'Mag': MAG, 'CSS images': CSS_Images,
                 'SDSS': SDSS, 'Others': OTHERS, 'Followed': FOLLOWED, 'Last': LAST,
                 'LC': LC, 'FC': FC, 'Classification': CLASS}
        data = {'HDU_Ext': HDU_EXT, 'Sequence': SEQUENCE, 'Date': DATE, 'Field_ID':FIELD_ID,
                'Obs_In_Seq': OBS_IN_SEQ, 'Cutout': CUTOUT}
        fitsio.write(filename, data, header=hdict)

        for file in files:
            img = np.load(path_file+file)
            fitsio.write(filename, img)
        return True
    else:
        print('{} Do not found in MANTRA'.format(ID))
        return False

In [ ]:
classes =  ['NON']
for classification in classes:
    path = '../mantra_data/{}/'.format(classification)
    objects = !ls $path
    for objct in objects:
        path_file = path+objct+'/'
        files = !ls $path_file
        ID = objct
        value = create_fits_TRAN(ID,files,path_file,classification)
        if value == False:
            print(path_file)